#                                                 Overview
This dataset contains every wholesale purchase of liquor in the State of Iowa by retailers for sale to individuals since January 1, 2012. The State of Iowa controls the wholesale distribution of liquor intended for retail sale, which means this dataset offers a complete view of retail liquor sales in the entire state. The dataset contains every wholesale order of liquor by all grocery stores, liquor stores, convenience stores, etc., with details about the store and location, the exact liquor brand and size, and the number of bottles ordered.

In addition to being an excellent dataset for analyzing liquor sales, this is a large and clean public dataset of retail sales data. It can be used to explore problems like stockout prediction, retail demand forecasting, and other retail supply chain problems.

In [39]:
# First, import the relevant modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import requests
import yfinance as yf
import sqlalchemy
from functools import reduce

import json
import requests
from collections import defaultdict

In [41]:
url = 'https://data.iowa.gov/resource/m3tr-qhgy.json'
r = requests.get(url)
json_data = r.json()

In [42]:
Iowa_Liquor_Sales_API = 'https://data.iowa.gov/resource/m3tr-qhgy.csv'
iowa_liquor_sales = pd.read_csv(Iowa_Liquor_Sales_API)
iowa_liquor_sales

,invoice_line_no,date,store,name,address,city,zipcode,store_location,county_number,county,...,itemno,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
0,S31296100034,2016-03-16T00:00:00.000,3762,Wine and Spirits Gallery,7690 HICKMAN RD,WINDSOR HEIGHTS,50322.0,NaN,77.0,Polk,...,80457,Ryan's Cream Liqueur,12,1000,7.23,10.85,1,10.85,1.00,0.26
1,INV-20599500022,2019-07-15T00:00:00.000,4320,Fareway Stores #470 / Perry,NaN,NaN,NaN,NaN,NaN,NaN,...,10008,Scoresby Rare Scotch,6,1750,10.50,15.75,6,94.50,10.50,2.77
2,INV-28403900139,2020-07-01T00:00:00.000,2643,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701.0,NaN,7.0,BLACK HAWK,...,15628,Jameson,6,1750,32.48,48.72,2,97.44,3.50,0.92
3,INV-20560900100,2019-07-11T00:00:00.000,3456,Quick Shop / Clear Lake,NaN,NaN,NaN,NaN,NaN,NaN,...,10009,Scoresby Rare Scotch,12,1000,6.74,10.11,2,20.22,2.00,0.52
4,INV-20784900043,2019-07-23T00:00:00.000,5151,IDA Liquor,"500, Hwy 175",Ida Grove,51445.0,NaN,47.0,IDA,...,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,6,1850,18.00,27.00,6,162.00,11.10,2.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,INV-28766900073,2020-07-16T00:00:00.000,5142,Smokin' Joe's #13 Tobacco and Liquor Outlet,2315 Mt Vernon Rd SE,Cedar Rapids,52303.0,NaN,57.0,Linn,...,45247,Paramount Gold Rum,12,1000,4.42,6.63,3,19.89,3.00,0.79
996,INV-20337700022,2019-07-01T00:00:00.000,3882,Kwik Shop #579 / Davenport,2805 Telegraph Rd,Davenport,52802.0,NaN,82.0,SCOTT,...,43024,Admiral Nelson Spiced Rum,24,375,2.74,4.11,10,41.10,3.75,0.99
997,INV-25356900010,2020-02-20T00:00:00.000,5135,Fareway Stores #155 / Huxley,911 Hwy 69,Huxley,50124.0,NaN,85.0,STORY,...,42716,Malibu Coconut Rum,12,750,7.49,11.24,12,134.88,9.00,2.37
998,INV-20490700100,2019-07-09T00:00:00.000,2666,Hy-Vee Food Store #2 / State Ankeny,NaN,NaN,NaN,NaN,NaN,NaN,...,43024,Admiral Nelson Spiced Rum,24,375,2.74,4.11,2,8.22,0.75,0.19


In [43]:
iowa_liquor_sales.head()

,invoice_line_no,date,store,name,address,city,zipcode,store_location,county_number,county,...,itemno,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
0,S31296100034,2016-03-16T00:00:00.000,3762,Wine and Spirits Gallery,7690 HICKMAN RD,WINDSOR HEIGHTS,50322.0,NaN,77.0,Polk,...,80457,Ryan's Cream Liqueur,12,1000,7.23,10.85,1,10.85,1.0,0.26
1,INV-20599500022,2019-07-15T00:00:00.000,4320,Fareway Stores #470 / Perry,NaN,NaN,NaN,NaN,NaN,NaN,...,10008,Scoresby Rare Scotch,6,1750,10.50,15.75,6,94.50,10.5,2.77
2,INV-28403900139,2020-07-01T00:00:00.000,2643,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701.0,NaN,7.0,BLACK HAWK,...,15628,Jameson,6,1750,32.48,48.72,2,97.44,3.5,0.92
3,INV-20560900100,2019-07-11T00:00:00.000,3456,Quick Shop / Clear Lake,NaN,NaN,NaN,NaN,NaN,NaN,...,10009,Scoresby Rare Scotch,12,1000,6.74,10.11,2,20.22,2.0,0.52
4,INV-20784900043,2019-07-23T00:00:00.000,5151,IDA Liquor,"500, Hwy 175",Ida Grove,51445.0,NaN,47.0,IDA,...,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,6,1850,18.00,27.00,6,162.00,11.1,2.93


In [44]:
iowa_liquor_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   invoice_line_no      1000 non-null   object 
 1   date                 1000 non-null   object 
 2   store                1000 non-null   int64  
 3   name                 1000 non-null   object 
 4   address              762 non-null    object 
 5   city                 762 non-null    object 
 6   zipcode              762 non-null    float64
 7   store_location       227 non-null    object 
 8   county_number        752 non-null    float64
 9   county               752 non-null    object 
 10  category             1000 non-null   int64  
 11  category_name        997 non-null    object 
 12  vendor_no            1000 non-null   int64  
 13  vendor_name          1000 non-null   object 
 14  itemno               1000 non-null   int64  
 15  im_desc              1000 non-null   ob

In [45]:
iowa_liquor_sales.describe()

,store,zipcode,county_number,category,vendor_no,itemno,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
count,1000.00000,762.000000,752.000000,1.000000e+03,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,3847.83500,51203.181102,59.389628,1.036496e+06,264.735000,39493.930000,12.29900,897.900000,10.033650,15.052400,9.907000,131.136120,8.683710,2.289170
std,1116.42909,1008.063791,25.706601,5.867247e+04,138.038891,59363.226262,7.65233,510.053423,7.749894,11.624899,17.295592,470.516239,21.153072,5.588366
min,2190.00000,50009.000000,2.000000,1.011100e+06,35.000000,258.000000,1.00000,50.000000,0.900000,1.350000,1.000000,2.700000,0.050000,0.010000
25%,2625.00000,50311.000000,47.000000,1.012100e+06,115.000000,23826.000000,6.00000,600.000000,5.400000,8.100000,3.000000,36.000000,1.750000,0.460000
50%,3848.50000,51040.000000,70.000000,1.031100e+06,260.000000,35318.000000,12.00000,750.000000,7.895000,11.845000,6.000000,66.960000,6.000000,1.580000
75%,4947.00000,52244.750000,78.000000,1.041100e+06,380.000000,40595.250000,12.00000,1000.000000,13.000000,19.500000,12.000000,135.000000,10.500000,2.770000
max,6062.00000,52807.000000,97.000000,1.901200e+06,626.000000,927618.000000,48.00000,1850.000000,99.990000,149.990000,300.000000,13806.000000,525.000000,138.690000
